In [ ]:
import math
import threading
import queue
from utils.thresholding import *

In [ ]:
def compute_lane_heading_angle(left_fit=None, right_fit=None, y_eval=200):
    """
    Computes the average heading angle (in degrees) between the vehicle's forward direction (image y-axis)
    and the tangent direction of the lane lines at the bottom of the image (y = y_eval).
    
    Returns a signed angle:
    - Positive → turn right
    - Negative → turn left
    """
    angles = []

    def angle_from_fit(fit):
        # Derivative of the 2nd-order polynomial at y = y_eval
        dy = 1.0  # pixel change in y
        dx = 2 * fit[0] * y_eval + fit[1]  # derivative at y_eval
        angle_rad = np.arctan(dx)  # slope to angle
        return -np.degrees(angle_rad)

    if left_fit is not None:
        angles.append(angle_from_fit(left_fit))
    if right_fit is not None:
        angles.append(angle_from_fit(right_fit))

    if angles:
        print(angles)
        # Average heading angle from both lanes
        return np.mean(angles)
    else:
        return None

In [ ]:
def detect_lane_lines_connected_components(binary_warped):
    # Ensure the input is binary (0 and 255)
    binary = np.uint8(binary_warped * 255) if binary_warped.max() <= 1 else np.uint8(binary_warped)

    # Apply connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)

    # Image dimensions
    height, width = binary.shape

    # Store points
    leftx, lefty, rightx, righty = [], [], [], []

    for i in range(1, num_labels):  # Label 0 is background
        x, y, w, h, area = stats[i]
        cx, cy = centroids[i]

        # Heuristic filters for likely lane lines
        if area > 1000 and h > 30:
            if cx < width // 2:
                coords = np.column_stack(np.where(labels == i))
                for pt in coords:
                    lefty.append(pt[0])
                    leftx.append(pt[1])
            else:
                coords = np.column_stack(np.where(labels == i))
                for pt in coords:
                    righty.append(pt[0])
                    rightx.append(pt[1])

    # Convert lists to numpy arrays
    leftx = np.array(leftx)
    lefty = np.array(lefty)
    rightx = np.array(rightx)
    righty = np.array(righty)

    return leftx, lefty, rightx, righty

In [ ]:
frame_in_w = 640
frame_in_h = 480

ym_per_pix = 0.1524 / 72.0
xm_per_pix = 0.2286 / 600.0
y_eval = 200
angle = 0

#ROI Parameters
y_bottom = frame_in_h
y_top = int(frame_in_h * 0.3)

In [ ]:
scale_factor_h = frame_in_w / 1280
scale_factor_v = frame_in_h / 720
offset = 200 * scale_factor_h
src = np.float32([
    [100, 100],
    [600, 100],
    [600, 390],
    [40, 390]
])
dst = np.float32([[offset, 0], [frame_in_w - offset, 0], [frame_in_w - offset, frame_in_h], [offset, frame_in_h]])
M = cv2.getPerspectiveTransform(src,dst)

In [ ]:
def capture_frames(videoIn, frame_queue):
    while True:
        ret, frame = videoIn.read()
        if not ret:
            print("Failed to grab frame.")
            break
        if frame_queue.full():
            # If the queue is full (only 1 frame), remove the oldest frame
            frame_queue.get()  # Discard the old frame to keep the queue size at 1
        frame_queue.put(frame)

In [ ]:
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
print("capture device is open: " + str(videoIn.isOpened()))

frame_queue = queue.Queue(maxsize=1)

capture_thread = threading.Thread(target=capture_frames, args=(videoIn, frame_queue))
capture_thread.daemon = True
capture_thread.start()

In [ ]:
while (True):
    if frame_queue.empty():
        continue
    
    # Get the most recent frame from the queue
    frame_vga = frame_queue.get()
    b_thresholded = threshold(frame_vga)
    binary_warped = cv2.warpPerspective(b_thresholded,M, (frame_in_w, frame_in_h))[y_top:y_bottom, :]
    kernel = np.ones((20, 20), np.uint8)

    # Clean small blobs
    binary_cleaned = cv2.morphologyEx(binary_warped, cv2.MORPH_OPEN, kernel)

    # Fill small gaps
    binary_cleaned = cv2.morphologyEx(binary_cleaned, cv2.MORPH_CLOSE, kernel)
    
    leftx, lefty, rightx, righty = detect_lane_lines_connected_components(binary_cleaned)
    
    if leftx.size > 0 and lefty.size > 0 and rightx.size > 0 and righty.size > 0:
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        angle = compute_lane_heading_angle(left_fit=left_fit, right_fit=right_fit, y_eval=y_eval)
    elif leftx.size > 0 and lefty.size > 0:
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = [0,0,0]

        angle = compute_lane_heading_angle(left_fit=left_fit, y_eval=y_eval)
    elif rightx.size > 0 and righty.size > 0:
        # Fit a second order polynomial to each
        left_fit = [0,0,0]
        right_fit = np.polyfit(righty, rightx, 2)

        angle = compute_lane_heading_angle(right_fit=right_fit, y_eval=y_eval)
    
    print(angle)